In [16]:
# import json module
import json
# because JSONDecodeError comes from the JSON module, we need to specifically import it as well
# import JSONEncoder to create our own JSON Encoder
from json import JSONDecodeError, JSONEncoder

## JSON

### Loading JSON

In [5]:
jsonString = '{"a": "apple", "b": "bear", "c": "cat"}'
json.loads(jsonString)
# result is a python dictionary

{'a': 'apple', 'b': 'bear', 'c': 'cat'}

In [8]:
# you can add a trailing comma to a python dictionary
{'a': 'apple', 'b': 'bear', 'c': 'cat',}

{'a': 'apple', 'b': 'bear', 'c': 'cat'}

In [9]:
# but you get a JSONDecodeError when you add a trailing comma to a json string
jsonString = '{"a": "apple", "b": "bear", "c": "cat",}'
json.loads(jsonString)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 40 (char 39)

In [12]:
# to catch this, use try-except, especially when working with JSON from a potentially untrustworthy source
jsonString = '{"a": "apple", "b": "bear", "c": "cat",}'
try:
    json.loads(jsonString)
except JSONDecodeError:
    print('Could not parse JSON!')

Could not parse JSON!


### Dumping JSON

In [13]:
# dump python dictionary into a JSON string
pythonDict = {'a': 'apple', 'b': 'bear', 'c': 'cat',}
json.dumps(pythonDict)

# you usually don't add any exception handling in this case because if you have a valid python dictionary,
# there's not a lot that could go wrong when you're formatting it as a JSON string

'{"a": "apple", "b": "bear", "c": "cat"}'

### Custom JSON Decoders

In [15]:
# but there is one exception to this rule where an exception could be thrown

class Animal:
    def __init__(self, name):
        self.name = name

# modify dictionary to use the Animal class
pythonDict = {'a': Animal('aardvark'), 'b': Animal('bear'), 'c': Animal('cat'),}
json.dumps(pythonDict)

# the JSON module has no idea how to handle the Animal class
# it doesn't know what the JSON equivalent should be 
# what we do here is override the default JSON encoder that it's using with a JSON encoder of our own

TypeError: Object of type Animal is not JSON serializable

In [17]:
class Animal:
    def __init__(self, name):
        self.name = name

# extend the JSONEncoder class to create a new AnimalEncoder
class AnimalEncoder(JSONEncoder):
    # the only thing we need to overwrite is the default method
    # o - is the object that's being passed in here that needs to be decoded into JSON
    def default(self, o):
        # if we're dealing with an Animal object
        if type(o) == Animal:
            return o.name
        # if it's not an Animal, we pass it off to the parent version of the encoder by using super(),
        #     which is the parent JSONEncoder class, and then calling that default method
        return super().default(o)
    
pythonDict = {'a': Animal('aardvark'), 'b': Animal('bear'), 'c': Animal('cat'),}
# we need to let json.dumps() know that we need to use the AnimalEncoder by passing the cls keyword argument
json.dumps(pythonDict, cls=AnimalEncoder)

# generally, you don't need to surround json.dumps() in a try-except statement because you're usually creating the dictionaries 
#     that you pass into json.dumps() in your own code, and you should appropriately handle any data types that you're passing in there

'{"a": "aardvark", "b": "bear", "c": "cat"}'